<a href="https://colab.research.google.com/github/newmantic/simplex/blob/main/simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np

class Simplex:
    def __init__(self, A, b, c):
        self.A = np.array(A)  # Coefficient matrix
        self.b = np.array(b)  # Right-hand side vector
        self.c = np.array(c)  # Cost vector

    def solve(self):
        num_vars = len(self.c)
        num_constraints = len(self.b)

        # Create the initial tableau
        tableau = np.hstack((self.A, np.eye(num_constraints), self.b.reshape(-1, 1)))
        tableau = np.vstack((tableau, np.hstack((-self.c, np.zeros(num_constraints), 0))))

        while True:
            # Check for optimality
            if all(tableau[-1, :-1] >= 0):
                break

            # Find entering variable (most negative coefficient in the last row)
            entering = np.argmin(tableau[-1, :-1])

            # Calculate the ratios, ignore divisions by zero or negative
            ratios = np.full(num_constraints, np.inf)
            for i in range(num_constraints):
                if tableau[i, entering] > 0:  # Only consider positive entries
                    ratios[i] = tableau[i, -1] / tableau[i, entering]

            leaving = np.argmin(ratios)

            # Perform pivoting
            pivot = tableau[leaving, entering]
            tableau[leaving] /= pivot  # Normalize the pivot row

            for i in range(len(tableau)):
                if i != leaving:
                    tableau[i] -= tableau[i, entering] * tableau[leaving]

        # Extract the solution
        solution = np.zeros(num_vars)
        for j in range(num_vars):
            # Check if variable is basic
            if np.any(tableau[:, j] == 1) and np.sum(tableau[:, j] == 1) == 1:
                row_index = np.where(tableau[:, j] == 1)[0][0]
                solution[j] = tableau[row_index, -1]

        optimal_value = tableau[-1, -1] * -1  # Negate to get the correct value
        return solution, optimal_value






In [12]:
# Example test cases
def additional_tests():
    # Test Case 1: Simple linear program
    A1 = [[1, 1], [2, 1], [1, 2]]
    b1 = [4, 8, 6]
    c1 = [3, 4]  # Maximize 3x + 4y
    print("Test Case 1:")
    simplex1 = Simplex(A1, b1, c1)
    solution1, optimal_value1 = simplex1.solve()
    print(f"Optimal Solution: {solution1}, Optimal Value: {optimal_value1}")

    # Test Case 2: Infeasible case
    A2 = [[1, 1], [1, -1], [1, 0]]
    b2 = [2, -1, 1]
    c2 = [1, 1]  # Maximize x + y
    print("Test Case 2:")
    simplex2 = Simplex(A2, b2, c2)
    try:
        solution2, optimal_value2 = simplex2.solve()
        print(f"Optimal Solution: {solution2}, Optimal Value: {optimal_value2}")
    except Exception as e:
        print(f"Error: {e}")

    # Test Case 3: Degenerate case
    A3 = [[1, 2], [2, 1], [1, 1]]
    b3 = [6, 6, 4]
    c3 = [2, 3]  # Maximize 2x + 3y
    print("Test Case 3:")
    simplex3 = Simplex(A3, b3, c3)
    solution3, optimal_value3 = simplex3.solve()
    print(f"Optimal Solution: {solution3}, Optimal Value: {optimal_value3}")


# Run the tests
test_simplex()
additional_tests()

Optimal Solution:
x = 4.0, y = 0.0
Optimal Value: -12.0
Test Case 1:
Optimal Solution: [2. 2.], Optimal Value: -14.0
Test Case 2:
Optimal Solution: [0.5 1.5], Optimal Value: -2.0
Test Case 3:
Optimal Solution: [2. 2.], Optimal Value: -10.0
